In [30]:
# %% [markdown]
# # Flight Difficulty Score Notebook (cell-separated)
# 
# Each cell below handles one part of the problem statement:
# - Data load & preprocess
# - EDA questions (each answered in their own cell)
# - Feature engineering
# - ML weight optimization
# - Flight Difficulty Score (daily normalization, ranking, classification)
# - Post-analysis & operational insights
# - Export results
# 
# Paste into a Jupyter notebook or a Python script (cells supported by `# %%`).

# %%
# Imports & settings
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 160)

In [31]:
airports = pd.read_csv("/kaggle/input/bashira/Airports Data.csv")            # columns: airport_iata_code, iso_country_code
flights = pd.read_csv("/kaggle/input/bashira/Flight Level Data.csv")        # your flight-level sample
pnr_remark = pd.read_csv("/kaggle/input/bashira/PNR Remark Level Data.csv") # record_locator, pnr_creation_date, flight_number, special_service_request
pnr_flight = pd.read_csv("/kaggle/input/bashira/PNRFlightLevelData.csv")    # pnr flight-level sample
bags = pd.read_csv("/kaggle/input/bashira/BagLevelData.csv")                # bag-level sample

print("Loaded tables:")
print(" airports:", airports.shape)
print(" flights:", flights.shape)
print(" pnr_remark:", pnr_remark.shape)
print(" pnr_flight:", pnr_flight.shape)
print(" bags:", bags.shape)

Loaded tables:
 airports: (5612, 2)
 flights: (8099, 15)
 pnr_remark: (51698, 4)
 pnr_flight: (687878, 12)
 bags: (687245, 8)


In [32]:
# Preprocess & canonical merge (one cell)
def safe_parse_datetime_col(df, col):
    if col in df.columns:
        return pd.to_datetime(df[col], errors='coerce')
    else:
        return pd.Series(pd.NaT, index=df.index)

# normalize date columns in flights
flights['scheduled_departure_datetime_local'] = safe_parse_datetime_col(flights, 'scheduled_departure_datetime_local')
flights['scheduled_arrival_datetime_local'] = safe_parse_datetime_col(flights, 'scheduled_arrival_datetime_local')
flights['actual_departure_datetime_local'] = safe_parse_datetime_col(flights, 'actual_departure_datetime_local')
flights['actual_arrival_datetime_local'] = safe_parse_datetime_col(flights, 'actual_arrival_datetime_local')

# ensure scheduled_departure_date_local exists as date (some inputs use different formats)
def extract_date_from_field(df, col_dt, col_date):
    if col_date in df.columns:
        # try to parse flexible formats
        parsed = pd.to_datetime(df[col_date], errors='coerce', dayfirst=False)
        if parsed.isna().mean() > 0.5:
            parsed = pd.to_datetime(df[col_date], errors='coerce', dayfirst=True)
        return parsed.dt.date
    elif col_dt in df.columns:
        return pd.to_datetime(df[col_dt], errors='coerce').dt.date
    else:
        return pd.Series([pd.NaT]*len(df), index=df.index)

flights['scheduled_departure_date_local'] = extract_date_from_field(flights, 'scheduled_departure_datetime_local', 'scheduled_departure_date_local')
pnr_flight['scheduled_departure_date_local'] = extract_date_from_field(pnr_flight, None, 'scheduled_departure_date_local')
bags['scheduled_departure_date_local'] = extract_date_from_field(bags, None, 'scheduled_departure_date_local')

# standardize keys to string
for df in (flights, pnr_flight, bags):
    if 'company_id' in df.columns:
        df['company_id'] = df['company_id'].astype(str).str.strip()
    if 'flight_number' in df.columns:
        df['flight_number'] = df['flight_number'].astype(str).str.strip()

# Create canonical 'flight_id' where possible
def make_fid(df):
    # safe creation: fillna with empty string
    return df['company_id'].fillna('') + '|' + df['flight_number'].fillna('') + '|' + df['scheduled_departure_date_local'].astype(str)
flights['flight_id'] = make_fid(flights)
pnr_flight['flight_id'] = make_fid(pnr_flight)
bags['flight_id'] = make_fid(bags)

# Aggregate PNR flight-level
if {'company_id','flight_number','scheduled_departure_date_local'}.issubset(pnr_flight.columns):
    pnr_agg = pnr_flight.groupby(['company_id','flight_number','scheduled_departure_date_local','flight_id']).agg(
        total_pax = ('total_pax', 'sum'),
        basic_economy_count = ('basic_economy_ind', lambda s: (s=='Y').sum() if s.dtype==object else s.sum()),
        lap_child_count = ('lap_child_count', 'sum'),
        stroller_users = ('is_stroller_user', lambda s: (s=='Y').sum() if s.dtype==object else s.sum())
    ).reset_index()
else:
    # fallback: aggregate on flight_id only
    pnr_agg = pnr_flight.groupby('flight_id').agg(
        total_pax = ('total_pax', 'sum'),
        basic_economy_count = ('basic_economy_ind', lambda s: (s=='Y').sum() if s.dtype==object else s.sum()),
        lap_child_count = ('lap_child_count', 'sum'),
        stroller_users = ('is_stroller_user', lambda s: (s=='Y').sum() if s.dtype==object else s.sum())
    ).reset_index()

# PNR remarks -> SSR counts: map record_locator -> flight_id using pnr_flight
if 'record_locator' in pnr_remark.columns and 'record_locator' in pnr_flight.columns:
    rl_map = pnr_flight[['record_locator','flight_id']].drop_duplicates()
    pnr_remark = pnr_remark.merge(rl_map, on='record_locator', how='left')
    ssr_agg = pnr_remark.groupby('flight_id').agg(ssr_count=('special_service_request','count')).reset_index()
else:
    # If can't link, use flight_number only (less reliable)
    if 'flight_number' in pnr_remark.columns:
        ssr_agg = pnr_remark.groupby('flight_number').size().reset_index(name='ssr_count')
        ssr_agg.rename(columns={'flight_number':'flight_number_for_ssr'}, inplace=True)
    else:
        ssr_agg = pd.DataFrame(columns=['flight_id','ssr_count'])

# Bags aggregation
bag_agg = bags.groupby('flight_id').agg(
    total_bags = ('bag_tag_unique_number', 'count'),
    transfer_bags = ('bag_type', lambda s: (s=='Transfer').sum()),
    origin_bags = ('bag_type', lambda s: (s=='Origin').sum())
).reset_index()

# Now merge everything onto flights using flight_id primarily
df = flights.merge(pnr_agg[['flight_id','total_pax','basic_economy_count','lap_child_count','stroller_users']], on='flight_id', how='left')
if 'flight_id' in ssr_agg.columns:
    df = df.merge(ssr_agg, on='flight_id', how='left')
else:
    # if ssr_agg has flight_number_for_ssr, merge using flight_number
    if 'flight_number_for_ssr' in ssr_agg.columns:
        df = df.merge(ssr_agg, left_on='flight_number', right_on='flight_number_for_ssr', how='left')
        df.drop(columns=['flight_number_for_ssr'], inplace=True)
bag_cols = ['total_bags','transfer_bags','origin_bags']
df = df.merge(bag_agg, on='flight_id', how='left')
df[['total_pax','basic_economy_count','lap_child_count','stroller_users','ssr_count','total_bags','transfer_bags','origin_bags']] = \
    df[['total_pax','basic_economy_count','lap_child_count','stroller_users','ssr_count','total_bags','transfer_bags','origin_bags']].fillna(0)

# Join airports to determine international flights
airports_cols = airports.rename(columns={'airport_iata_code':'scheduled_arrival_station_code','iso_country_code':'arrival_country'})
if 'scheduled_arrival_station_code' in df.columns:
    df = df.merge(airports_cols[['scheduled_arrival_station_code','arrival_country']].drop_duplicates(),
                  on='scheduled_arrival_station_code', how='left')
    df['is_international'] = (df['arrival_country'] != 'US').astype(int)
else:
    df['is_international'] = 0

# ensure numeric columns
for col in ['total_seats','scheduled_ground_time_minutes','actual_ground_time_minutes','minimum_turn_minutes']:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

print("Merged flights rows:", len(df))

Merged flights rows: 8099


In [33]:
# EDA Q1 — Average delay & % flights depart later than scheduled
# (One cell dedicated to this deliverable)
df['dep_delay_min'] = (df['actual_departure_datetime_local'] - df['scheduled_departure_datetime_local']).dt.total_seconds() / 60
avg_delay_overall = df['dep_delay_min'].dropna().mean()
pct_depart_late = (df['dep_delay_min'] > 0).mean() * 100

print("EDA Q1: Average departure delay (minutes):", round(avg_delay_overall,2))
print("EDA Q1: Percentage of flights departing later than scheduled: {:.2f}%".format(pct_depart_late))

EDA Q1: Average departure delay (minutes): 21.18
EDA Q1: Percentage of flights departing later than scheduled: 49.61%


In [34]:
# EDA Q2 — How many flights have scheduled ground time close to or below the minimum turn mins?
# (One cell dedicated to this deliverable)
df['ground_time_cushion'] = df['scheduled_ground_time_minutes'] - df['minimum_turn_minutes']
# count flights with cushion <= 5 minutes (tunable)
threshold = 5
tight_turns_count = ((df['ground_time_cushion'] <= threshold) & (~df['ground_time_cushion'].isna())).sum()
print("EDA Q2: Flights with scheduled ground time within {} mins of min turn: {} (out of {})".format(threshold, tight_turns_count, len(df)))


EDA Q2: Flights with scheduled ground time within 5 mins of min turn: 780 (out of 8099)


In [35]:
# EDA Q3 — Average ratio of transfer bags vs checked bags across flights
# (One cell dedicated to this deliverable)
# define checked bags = total_bags ; interpret transfer ratio = transfer_bags / total_bags
df['transfer_bag_ratio'] = np.where(df['total_bags']>0, df['transfer_bags'] / df['total_bags'], np.nan)
avg_transfer_ratio = df['transfer_bag_ratio'].dropna().mean()
print("EDA Q3: Average transfer bag ratio (transfer/total) across flights (ignoring flights with 0 bags):",
      "{:.2%}".format(avg_transfer_ratio) if not np.isnan(avg_transfer_ratio) else "No bag data")


EDA Q3: Average transfer bag ratio (transfer/total) across flights (ignoring flights with 0 bags): 51.52%


In [36]:
# EDA Q4 — How do passenger loads compare and correlation with operational difficulty?
# (One cell dedicated to this deliverable)
df['load_factor'] = np.where(df['total_seats']>0, df['total_pax'] / df['total_seats'], np.nan)
# Show distribution summary
print("EDA Q4: Load factor distribution (describe):")
print(df['load_factor'].describe())

# Correlate load factor with departure delay using Spearman (robust to non-linearity)
corr_spearman = df[['load_factor','dep_delay_min']].corr(method='spearman').iloc[0,1]
print("EDA Q4: Spearman correlation between load factor and departure delay (min):", round(corr_spearman,3))


EDA Q4: Load factor distribution (describe):
count    8099.000000
mean        1.024236
std         0.209837
min         0.060000
25%         0.940000
50%         1.047619
75%         1.145251
max         1.895000
Name: load_factor, dtype: float64
EDA Q4: Spearman correlation between load factor and departure delay (min): -0.005


In [37]:
# EDA Q5 — Are high SSR flights also high-delay after controlling for load?
# (One cell dedicated to this deliverable)
# We'll run a simple grouped comparison: within load bins, compare avg delay for low vs high SSR flights.
df['load_bin'] = pd.qcut(df['load_factor'].rank(method='first'), q=3, labels=['Low','Medium','High'])
# SSR bin: low/high by median split (robust for small samples)
df['ssr_bin'] = pd.cut(df['ssr_count'], bins=[-0.1, 0, df['ssr_count'].quantile(0.75), df['ssr_count'].max()+1],
                       labels=['NoSSR','LowSSR','HighSSR'])
# compute average delay per bin
ssr_delay_table = df.groupby(['load_bin','ssr_bin'])['dep_delay_min'].mean().unstack()
print("EDA Q5: Average departure delay (min) by load_bin vs ssr_bin:")
print(ssr_delay_table.fillna(0))

# Quick regression-like check controlling for load:
from statsmodels.formula.api import ols
# create small dataframe with needed columns and dropna
reg_df = df[['dep_delay_min','ssr_count','load_factor']].dropna()
if len(reg_df) > 5:
    model = ols("dep_delay_min ~ ssr_count + load_factor", data=reg_df).fit()
    print("\nRegression controlling for load_factor:\n", model.summary().tables[1])
else:
    print("\nNot enough data to run regression for EDA Q5.")

EDA Q5: Average departure delay (min) by load_bin vs ssr_bin:
ssr_bin       NoSSR     LowSSR    HighSSR
load_bin                                 
Low       24.806109  31.519380  40.030075
Medium    14.759259  14.444444  22.540541
High      14.854342  14.585492  20.103880

Regression controlling for load_factor:
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      66.3261      3.396     19.532      0.000      59.670      72.983
ssr_count       0.5780      0.091      6.375      0.000       0.400       0.756
load_factor   -47.9453      3.292    -14.565      0.000     -54.398     -41.493


In [38]:
# Feature engineering (one cell)
# Build the features to use in FDS and ML
df_feat = df.copy()

# Time features
df_feat['scheduled_ground_time_minutes'] = pd.to_numeric(df_feat['scheduled_ground_time_minutes'], errors='coerce').fillna(0)
df_feat['actual_ground_time_minutes'] = pd.to_numeric(df_feat['actual_ground_time_minutes'], errors='coerce').fillna(0)
df_feat['minimum_turn_minutes'] = pd.to_numeric(df_feat['minimum_turn_minutes'], errors='coerce').fillna(0)
df_feat['scheduled_ground_margin'] = df_feat['scheduled_ground_time_minutes'] - df_feat['minimum_turn_minutes']

# baggage features
df_feat['total_bags'] = df_feat['total_bags'].fillna(0)
df_feat['transfer_bag_count'] = df_feat['transfer_bags'].fillna(0)
df_feat['origin_bag_count'] = df_feat['origin_bags'].fillna(0)
df_feat['bags_per_pax'] = np.where(df_feat['total_pax']>0, df_feat['total_bags']/df_feat['total_pax'], 0)
df_feat['transfer_bag_ratio'] = np.where(df_feat['total_bags']>0, df_feat['transfer_bag_count']/df_feat['total_bags'], 0)

# passenger features (already have total_pax, basic_economy_count, stroller_users, lap_child_count)
df_feat['basic_economy_count'] = df_feat['basic_economy_count'].fillna(0).astype(int)
df_feat['stroller_users'] = df_feat['stroller_users'].fillna(0).astype(int)
df_feat['lap_child_count'] = df_feat['lap_child_count'].fillna(0).astype(int)

# SSR
df_feat['ssr_count'] = df_feat['ssr_count'].fillna(0).astype(int)

# aircraft size
df_feat['total_seats'] = df_feat['total_seats'].fillna(0).astype(int)

# target for ML: dep_delay_min (already computed in df)
df_feat['dep_delay_min'] = df_feat['dep_delay_min'].fillna(0)

print("Features engineered: columns sample ->", df_feat[['scheduled_ground_margin','transfer_bag_ratio','bags_per_pax','total_pax','ssr_count','load_factor']].head())


Features engineered: columns sample ->    scheduled_ground_margin  transfer_bag_ratio  bags_per_pax  total_pax  ssr_count  load_factor
0                        8            0.476190      0.646154         65         14     0.855263
1                       90            0.258741      0.836257        171          6     1.023952
2                       25            0.447059      0.472222        180          1     1.084337
3                      194            0.740741      0.981818         55          8     1.100000
4                       24            0.728571      0.514706        136         10     0.819277


In [39]:
# ML weight optimization (one cell)
# Use RandomForestRegressor to learn importances to predict dep_delay_min
from sklearn.ensemble import RandomForestRegressor
feature_cols = ['dep_delay_min','scheduled_ground_margin','transfer_bag_ratio','bags_per_pax','total_pax','ssr_count','load_factor']
# Note: we should not include the target dep_delay_min as a predictor — remove it for training
train_features = ['scheduled_ground_margin','transfer_bag_ratio','bags_per_pax','total_pax','ssr_count','load_factor','is_international']
# Prepare training set — only flights with non-null features
train_df = df_feat[train_features + ['dep_delay_min']].dropna()

weights_dict = {}
if len(train_df) >= 10:
    X = train_df[train_features].values
    y = train_df['dep_delay_min'].values
    rf = RandomForestRegressor(n_estimators=300, random_state=42)
    rf.fit(X,y)
    # feature importances
    importances = rf.feature_importances_
    # normalize to sum to 1
    importances_norm = importances / importances.sum() if importances.sum() else np.ones_like(importances)/len(importances)
    weights_dict = dict(zip(train_features, importances_norm))
    print("ML-optimized weights (learned from dep_delay_min):")
    for k,v in weights_dict.items():
        print(f"  {k}: {v:.4f}")
    # show R2 on training data (in-sample)
    preds = rf.predict(X)
    print("In-sample R2:", round(r2_score(y,preds),3))
else:
    # fallback equal weights
    print("Not enough data for ML optimization; using equal weights fallback.")
    w = 1.0/len(train_features)
    weights_dict = {f:w for f in train_features}
    print(weights_dict)

ML-optimized weights (learned from dep_delay_min):
  scheduled_ground_margin: 0.3718
  transfer_bag_ratio: 0.1483
  bags_per_pax: 0.1808
  total_pax: 0.1030
  ssr_count: 0.0711
  load_factor: 0.1149
  is_international: 0.0101
In-sample R2: 0.894


In [40]:
# Flight Difficulty Score computation per day (one cell)
# We'll normalize features per day and compute a weighted sum using ML weights
scaler = MinMaxScaler()
norm_cols = train_features.copy()
# ensure columns exist
for c in norm_cols:
    if c not in df_feat.columns:
        df_feat[c] = 0

# per-day normalization: min-max inside each day
def per_day_minmax(group, cols):
    res = group.copy()
    for c in cols:
        vals = res[c].values.astype(float)
        if len(vals) <= 1 or np.nanmax(vals)==np.nanmin(vals):
            res[c + '_norm'] = 0.0
        else:
            res[c + '_norm'] = (vals - np.nanmin(vals)) / (np.nanmax(vals) - np.nanmin(vals))
    return res

df_normed = df_feat.groupby('scheduled_departure_date_local', group_keys=False).apply(lambda g: per_day_minmax(g, norm_cols))

# Build weighted sum using weights_dict; if any missing weight, fallback to equal
if not weights_dict:
    weights_dict = {f:1/len(norm_cols) for f in norm_cols}
# Map weights for normalized columns
weights_for_norm = {c + '_norm': weights_dict.get(c, 1.0/len(norm_cols)) for c in norm_cols}

# compute raw_score
df_normed['raw_score'] = 0.0
for norm_col, w in weights_for_norm.items():
    df_normed['raw_score'] += df_normed[norm_col] * w

# scale to 0-100
df_normed['FDS'] = MinMaxScaler().fit_transform(df_normed[['raw_score']]) * 100

# Ranking & classification per day
df_normed['daily_rank'] = df_normed.groupby('scheduled_departure_date_local')['FDS'].rank(method='dense', ascending=False).astype(int)
# quantile-based categories (tertiles)
def tertile_labels(x):
    try:
        return pd.qcut(x.rank(method='first'), 3, labels=['Easy','Medium','Difficult'])
    except ValueError:
        # fallback: equal labels if too few values
        return pd.Series(['Medium']*len(x), index=x.index)
df_normed['category'] = df_normed.groupby('scheduled_departure_date_local')['FDS'].transform(lambda x: tertile_labels(x))

print("Computed FDS for {} flights.".format(len(df_normed)))

Computed FDS for 8099 flights.


In [41]:
# Deliverable outputs — Ranking & Classification (one cell)
# For each day: top difficult flights
days = df_normed['scheduled_departure_date_local'].dropna().unique()
for d in sorted(days):
    sub = df_normed[df_normed['scheduled_departure_date_local']==d].sort_values('FDS', ascending=False)
    print("\nDate:", d, "- Top 5 difficult flights")
    display_cols = ['company_id','flight_number','scheduled_arrival_station_code','total_pax','FDS','daily_rank','category','scheduled_ground_time_minutes','minimum_turn_minutes','transfer_bag_count','ssr_count']
    print(sub[display_cols].head(5).to_string(index=False))


Date: 2025-08-01 - Top 5 difficult flights
company_id flight_number scheduled_arrival_station_code  total_pax       FDS  daily_rank  category  scheduled_ground_time_minutes  minimum_turn_minutes  transfer_bag_count  ssr_count
        UA           909                            AMS        273 98.750407           1 Difficult                           3160                    95                 153         24
        UA           247                            OMA        147 96.269023           2 Difficult                           3910                   110                  74         12
        UA           881                            HND        394 78.393902           3 Difficult                           1230                   155                 262         16
        UA           972                            BRU        357 76.790321           4 Difficult                            691                   100                 276         22
        UA           953                 

In [42]:
# Post-analysis — which destinations consistently show more difficulty (one cell)
dest_stats = df_normed.groupby('scheduled_arrival_station_code').agg(
    mean_FDS=('FDS','mean'),
    median_FDS=('FDS','median'),
    count=('FDS','count')
).reset_index().sort_values('mean_FDS', ascending=False)
print("Top destinations by mean FDS (only show >=1 occurrences):")
print(dest_stats.head(10).to_string(index=False))

# Drivers analysis for top destinations
top_dests = dest_stats[dest_stats['count']>=1].head(5)['scheduled_arrival_station_code'].tolist()
drivers = df_normed[df_normed['scheduled_arrival_station_code'].isin(top_dests)].groupby('scheduled_arrival_station_code')[['scheduled_ground_margin_norm','transfer_bag_ratio_norm','bags_per_pax_norm','ssr_count_norm']].mean()
# Note: we may not have *_norm names for all drivers; attempt safe print
print("\nDriver averages for top destinations (names may vary if specific norm cols missing):")
print(drivers.fillna(0).to_string())

# If normalized driver cols aren't present (naming differences), compute simple averages:
driver_cols_fallback = ['scheduled_ground_margin','transfer_bag_ratio','bags_per_pax','ssr_count']
drivers_fallback = df_normed[df_normed['scheduled_arrival_station_code'].isin(top_dests)].groupby('scheduled_arrival_station_code')[driver_cols_fallback].mean()
print("\nDriver averages (fallback raw values):")
print(drivers_fallback.to_string())

Top destinations by mean FDS (only show >=1 occurrences):
scheduled_arrival_station_code  mean_FDS  median_FDS  count
                           BRU 64.932851   61.069846     15
                           GRU 60.747750   58.436203     15
                           HND 59.314308   57.364425     18
                           FRA 56.921716   56.140569     30
                           AMS 54.196521   47.842781     17
                           DUB 50.473599   44.034491     14
                           ATH 50.318268   52.008978     15
                           HNL 49.723052   48.998406     15
                           MUC 48.824948   47.122118     15
                           FAI 48.525162   46.752530     15

Driver averages for top destinations (names may vary if specific norm cols missing):
                                scheduled_ground_margin_norm  transfer_bag_ratio_norm  bags_per_pax_norm  ssr_count_norm
scheduled_arrival_station_code                                             

In [43]:
# Actionable recommendations (one cell)
print("\nRecommendations:")
print("1) For flights flagged as 'Difficult' (top tercile): pre-stage extra baggage handlers and gate agents; prioritize them for equipment/staging.")
print("2) For flights with scheduled_ground_margin <= minimum_turn_minutes + 5: flag as tight-turn; consider schedule buffer or crew standby.")
print("3) For high transfer_bag_ratio flights: assign transfer belt & interline team in advance.")
print("4) For flights with high ssr_count: pre-assign wheelchair and customer-service staff and add extra boarding time.")
print("5) Use daily FDS top-N (e.g., top 10 flights by FDS) as input to morning staffing rosters.")



Recommendations:
1) For flights flagged as 'Difficult' (top tercile): pre-stage extra baggage handlers and gate agents; prioritize them for equipment/staging.
2) For flights with scheduled_ground_margin <= minimum_turn_minutes + 5: flag as tight-turn; consider schedule buffer or crew standby.
3) For high transfer_bag_ratio flights: assign transfer belt & interline team in advance.
4) For flights with high ssr_count: pre-assign wheelchair and customer-service staff and add extra boarding time.
5) Use daily FDS top-N (e.g., top 10 flights by FDS) as input to morning staffing rosters.


In [44]:
# Save outputs (one cell)
out_cols = ['flight_id','company_id','flight_number','scheduled_departure_date_local','scheduled_arrival_station_code',
            'total_pax','total_seats','FDS','daily_rank','category','dep_delay_min','scheduled_ground_margin',
            'transfer_bag_count','ssr_count','load_factor']
df_normed[out_cols].to_csv("flight_difficulty_scores_by_flight.csv", index=False)
print("\nSaved flight_difficulty_scores_by_flight.csv")

# %% [markdown]
# End of notebook.
# 
# Notes:
# - This notebook preserves your original pipeline and adds ML weight optimization.
# - Each deliverable from the problem statement is implemented in its own cell with printed outputs.
# - If any cell errors due to missing columns in your real CSVs, check column names / capitalization and adjust the code accordingly.
# - To further stabilize weights, replace the single RandomForest fit with cross-validated averaging (I can add that on request).


Saved flight_difficulty_scores_by_flight.csv
